In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres. 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-26 17:59:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.61MB/s    in 0.2s    

2022-03-26 17:59:56 (5.61 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Combined-Weather-Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load combined weather data into Spark DataFrame
from pyspark import SparkFiles
url = "https://grp2winereviews.s3.amazonaws.com/cleaned_winedata.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+---+----------+----+------------------+------+
| id|  province|year|           variety|points|
+---+----------+----+------------------+------+
|  0|   Sicilia|2013|       White Blend|    87|
|  2|    Oregon|2013|        Pinot Gris|    87|
|  4|    Oregon|2012|        Pinot Noir|    87|
|  6|   Sicilia|2013|          Frappato|    87|
|  7|    Alsace|2012|    Gewürztraminer|    87|
|  9|    Alsace|2012|        Pinot Gris|    87|
| 10|California|2011|Cabernet Sauvignon|    87|
| 11|    Alsace|2012|    Gewürztraminer|    87|
| 12|California|2012|Cabernet Sauvignon|    87|
| 13|   Sicilia|2012| Nerello Mascalese|    87|
| 14|California|2012|        Chardonnay|    87|
| 21|    Oregon|2013|        Pinot Noir|    87|
| 22|   Sicilia|2007|       White Blend|    87|
| 23|California|2011|            Merlot|    87|
| 24|   Sicilia|2009|      Nero d'Avola|    87|
| 25|California|2011|        Pinot Noir|    87|
| 26|   Sicilia|2013|       White Blend|    87|
| 27|   Sicilia|2013|      Nero d'Avola|

In [5]:
# Configure settings for RDS - Group2Wineos password isn't an important password and was kept simple for this exercise.
mode = "append"
jdbc_url="jdbc:postgresql://grp2rdsinstance.cwkbfcctxu7y.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "Group2Wineos", 
          "driver":"org.postgresql.Driver"}

In [6]:
# Write df to table in RDS
df.write.jdbc(url=jdbc_url, table='wine_data', mode=mode, properties=config)